# Azure ML Pipelines

Цель лабораторной работы: 

- создание с __Конвейеров машинного обучения__ (ML Pipelines) в Azure ML
- тренировка модели машинного обучения.

## Подготовка 

Импорт необходимых модулей и проверка версии AzureML SDK:

In [24]:
import azureml.core
from azureml.core import Workspace, Environment, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.estimator import Estimator
from azureml.train.sklearn import SKLearn

# Check core SDK version number
print(f'SDK version: {azureml.core.VERSION}')

SDK version: 1.12.0


Зададим параметры Эксперимента:

In [25]:
experiment_name = 'ml_pipeline_demo'

experiment_dir = 'ml-pipeline-demo'
os.makedirs(experiment_dir, exist_ok=True)

## Соединение со Azure ML Workspace

Устанавливаем соединение с Рабочей областью в Azure ML:

In [26]:
ws = Workspace.from_config()
print(f'Successfully connected to Workspace: {ws.name}.')

Successfully connected to Workspace: ai-in-cloud-workspace.


## Зададим Шаги ML Конвейера

Зададим и сохраним скрипт для Шага регистрации модели:

In [35]:
%%writefile scripts/register-model.py

#%% Import libraries
import argparse
import joblib
from azureml.core import Workspace, Model, Run


#%% Get parameters
parser = argparse.ArgumentParser()
parser.add_argument('--model_dir', type=str, dest='model_dir', default='outputs', help='Define ML model directory')
args = parser.parse_args()


#%% Set model directory
model_dir = args.model_dir


#%% Get the experiment run context
run = Run.get_context()


#%% Load the model
model_path = f'{model_dir}/model.pkl'

print(f'Loading {model_path}...')
model = joblib.load(model_path)


#%% Register model
Model.register(workspace=run.experiment.workspace,
               model_path=model_path,
               model_name='diabetes_predict_model',
               tags={'Demo': 'ML Pipeline'})

#%% Finish Experiment
run.complete()


Overwriting scripts/register-model.py


## Создадим и запустим ML Конвейер

Подготовка к созданию:

In [28]:
# Get dataset
data_ds = ws.datasets.get('diabetes_db')
print(f'Used dataset {data_ds.name}: {data_ds.description}')

# Get environment
env = Environment.get(ws, 'diabetes-experiment-env')
print(f'Environment {env.name} will be used.')

# Get compute cluster
cluster = ComputeTarget(workspace=ws, name='ml-cluster')
cluster_state = cluster.get_status()
print(f'ML cluster contains {cluster_state.current_node_count} compute nodes in {cluster_state.allocation_state} state')

Used dataset diabetes_db: Diabetes Disease Database
Environment diabetes-experiment-env will be used.
ML cluster contains 1 compute nodes in Steady state


Скопируем необходимые скрипты в директорию Эксперимента:

In [36]:
!cp scripts/train-model.py $experiment_dir
!cp scripts/register-model.py $experiment_dir
!ls $experiment_dir

register-model.py  train-model.py


Определение Шагов ML Конвейера:

In [37]:
## Import packages related with Azure ML Pipeline 
from azureml.pipeline.core import Pipeline
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.core.runconfig import RunConfiguration


## Step 0: Prepare
# Set Pipeline Run Configuration
pipeline_run_config = RunConfiguration()
pipeline_run_config.target = cluster
pipeline_run_config.environment = env

# Create a PipelineData (temporary Data Reference) for the model folder
model_dir = PipelineData('model_dir', datastore=ws.get_default_datastore())


## Step 1: Train model
estimator = SKLearn(source_directory=experiment_dir,
                    compute_target=cluster,
                    environment_definition=pipeline_run_config.environment,
                    entry_script='train-model.py')


train_model_step = EstimatorStep(name='Train Model Step',
                                 estimator=estimator, 
                                 estimator_entry_script_arguments=['--output_dir', model_dir],
                                 inputs=[data_ds.as_named_input('data')],
                                 outputs=[model_dir],
                                 compute_target=cluster,
                                 allow_reuse=True)

## Step 2: Register trained model
register_model_step = PythonScriptStep(name='Register Model Step',
                                       source_directory=experiment_dir,
                                       script_name='register-model.py',
                                       arguments=['--model_dir', model_dir],
                                       inputs=[model_dir],
                                       compute_target=cluster,
                                       runconfig=pipeline_run_config,
                                       allow_reuse=True)

## Complete
print('Pipeline steps were completely defined.')

WARNING - If environment_definition or conda_dependencies_file_path is specified, Azure ML will not install any framework related packages on behalf of the user.


Pipeline steps were completely defined.


Запуск созданного ML-конвейера:

In [38]:
# Construct the pipeline
pipeline_steps = [train_model_step, register_model_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print("Pipeline was built.")

# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name=experiment_name
                       )
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")

pipeline_run.wait_for_completion(show_output=True)

Pipeline was built.
Created step Train Model Step [8908b48c][408b5cdb-d800-43a1-a26d-872da5598239], (This step will run and generate new outputs)
Created step Register Model Step [8512695e][d1793742-7892-4d54-bb44-2ae17611594a], (This step will run and generate new outputs)
Submitted PipelineRun e624a891-a35f-4d8f-976c-4822beb459bb
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml_pipeline_demo/runs/e624a891-a35f-4d8f-976c-4822beb459bb?wsid=/subscriptions/9aef4ce1-e591-4870-9443-0b0eb98df2aa/resourcegroups/ai-in-cloud-workshop-rg/workspaces/ai-in-cloud-workspace
Pipeline submitted for execution.
PipelineRunId: e624a891-a35f-4d8f-976c-4822beb459bb
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/ml_pipeline_demo/runs/e624a891-a35f-4d8f-976c-4822beb459bb?wsid=/subscriptions/9aef4ce1-e591-4870-9443-0b0eb98df2aa/resourcegroups/ai-in-cloud-workshop-rg/workspaces/ai-in-cloud-workspace
PipelineRun Status: NotStarted
PipelineRun Status: Running



'Finished'

_Перейдети по ссылкам, которые будут появляться в `output` выполнения ML Конвейра, чтобы проследить за статусом исполнения._

Получим список всех зарегистрированных ML моделей и найдем модель, обученную в этом Эксперименте: 

In [39]:
for model in Model.list(ws):
    print(f'{model.name} v{model.version}')
    
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t', tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t', prop_name, ':', prop)

diabetes_predict_model v2
	 Demo : ML Pipeline
diabetes_predict_model v1
	 Demo : Target compute
	 AUC : 0.846851712258014
	 Accuracy : 0.7788888888888889
diabetes_model v4
	 Dataset : Diabetes
	 AUC : 0.846851712258014
	 Accuracy : 0.7788888888888889
diabetes_model v3
	 Dataset : Diabetes
	 AUC : 0.846851712258014
	 Accuracy : 0.7788888888888889
diabetes_model v2
	 Dataset : Diabetes
	 AUC : 0.8468519356081545
	 Accuracy : 0.7788888888888889
diabetes_model v1
	 Training context : Estimator
	 AUC : 0.8468519356081545
	 Accuracy : 0.7788888888888889
amlstudio-covid19-service v1
	 CreatedByAMLStudio : true
amlstudio-covid19-service-pipe v1
	 CreatedByAMLStudio : true
amlstudio-covid19-spread-servi v1
	 CreatedByAMLStudio : true
amlstudio-pima-diabets-service v2
	 CreatedByAMLStudio : true
amlstudio-letter-recognition-s v1
	 CreatedByAMLStudio : true
amlstudio-pima-diabetes-model v1
	 CreatedByAMLStudio : true


Это простой пример, призванный продемонстрировать принцип. На самом деле вы можете встроить более сложную логику в этапы конвейера - например, оценить модель по некоторым тестовым данным, чтобы вычислить метрику производительности, такую как AUC или точность, сравнить метрику с метрикой любых ранее зарегистрированных версий модели и зарегистрировать новую модель только в том случае, если она работает лучше.

## Вывод

## Полезные ссылки
